<a href="https://colab.research.google.com/github/leegang/AI-Scientist/blob/main/Fork_of_TRAIN_CNN_CMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

cmi_detect_behavior_with_sensor_data_path = kagglehub.competition_download('cmi-detect-behavior-with-sensor-data')

print('Data source import complete.')


100%|██████████| 178M/178M [00:01<00:00, 107MB/s]

Extracting files...


Data source import complete.


In [4]:
!pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 45.7 MB/s eta 0:00:00


### CELL1

In [5]:
# ====================================================
# 1. SETUP AND CONFIGURATION
# ====================================================
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import gc
import polars as pl

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import f1_score,accuracy_score
import joblib

### CEll 2

In [6]:
# ====================================================
# 2. CONFIGURATION
# ====================================================
class CFG:
    # 数据路径
    DATA_DIR = "/root/.cache/kagglehub/competitions/cmi-detect-behavior-with-sensor-data/"
    TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
    TRAIN_DEMO_CSV = os.path.join(DATA_DIR, "train_demographics.csv")

    # 预处理数据路径
    PREPROCESSED_DIR = "/kaggle/working/preprocessed_data/"
    RUN_PREPROCESSING = not os.path.exists(PREPROCESSED_DIR) # 检查目录是否存在

    # 时序数据参数
    TARGET_SEQ_LEN = 512

    # 训练参数
    # MODEL_NAME = 'Custom1DCNN'
    N_FOLDS = 6
    BATCH_SIZE = 16
    EPOCHS =  60
    LEARNING_RATE =  1e-4
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    HIERARCHICAL_LOSS_ALPHA = 0.1

    # 输出路径
    MODEL_OUTPUT_DIR = "/kaggle/working/models/"

    # 特征列
    DEMO_COLS = ['adult_child', 'age', 'sex', 'handedness',
                 'height_cm', 'shoulder_to_wrist_cm', 'elbow_to_wrist_cm',
                 'shoulder_wrist_ratio', 'elbow_wrist_ratio']

# 创建必要的目录
os.makedirs(CFG.MODEL_OUTPUT_DIR, exist_ok=True)

### CELL3

In [ ]:
# ====================================================================
# CELL: PRE-PROCESSING & FEATURE ENGINEERING (OPTIMIZED & ENHANCED)
# ====================================================================

import polars as pl
import pandas as pd
import numpy as np
from scipy.spatial.transform import Rotation as R
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib
import json
import os
import shutil
from tqdm.auto import tqdm
from joblib import Parallel, delayed


def calculate_angular_velocity_vectorized(quat_values):
    """向量化版本：从四元数计算角速度。"""
    # 计算四元数的时间差分
    # q_t * conj(q_{t-1})
    q_diff = np.zeros_like(quat_values)

    # 填充第一个时间步的差分（保持为0）
    q_diff[1:] = R.from_quat(quat_values[1:]) * R.from_quat(quat_values[:-1]).inv()

    # 从差分四元数中提取旋转向量（其模长是角度，方向是轴）
    # 角速度约等于 旋转向量 / dt。这里我们假设 dt=1，所以角速度就是旋转向量。
    angular_velocity = q_diff.as_rotvec()

    return angular_velocity

# --- 辅助函数 ---

def remove_gravity_from_acc_vectorized(acc_values, quat_values):
    """
    向量化版本: 从加速度数据中移除重力分量。
    在整个数据集的Numpy数组上一次性操作。
    """
    num_samples = acc_values.shape[0]
    linear_accel = np.zeros_like(acc_values)
    gravity_world = np.array([0, 0, 9.81]) # 标准重力加速度

    # 识别有效的四元数行 (非NaN, 非零)
    valid_quat_mask = ~np.all(np.isnan(quat_values), axis=1) & ~np.all(np.isclose(quat_values, 0), axis=1)

    # 默认情况下，线性加速度等于原始加速度
    linear_accel = acc_values.copy()

    if np.any(valid_quat_mask):
        valid_acc = acc_values[valid_quat_mask]
        valid_quat = quat_values[valid_quat_mask]

        try:
            # Scipy可以批量处理四元数，效率极高
            rotations = R.from_quat(valid_quat)
            gravity_sensor_frame = rotations.apply(gravity_world, inverse=True)
            linear_accel[valid_quat_mask] = valid_acc - gravity_sensor_frame
        except ValueError as e:
            print(f"Warning: A portion of quaternion conversion failed: {e}. Falling back for those rows.")
            # 如果批量处理失败（例如，由于无效的四元数值），可以回退到循环处理
            # 但在大多数情况下，Scipy的批量处理是稳健的

    return linear_accel

def euler_from_quat_vectorized(quat_values):
    """向量化版本: 将四元数转换为欧拉角 (roll, pitch, yaw)。"""
    euler_angles = np.zeros((quat_values.shape[0], 3))

    valid_quat_mask = ~np.all(np.isnan(quat_values), axis=1) & ~np.all(np.isclose(quat_values, 0), axis=1)

    if np.any(valid_quat_mask):
        valid_quat = quat_values[valid_quat_mask]
        try:
            rotations = R.from_quat(valid_quat)
            # as_eul('xyz', degrees=False) 返回 roll, pitch, yaw (弧度)
            euler_angles[valid_quat_mask] = rotations.as_euler('xyz', degrees=False)
        except ValueError:
            print("Warning: Euler angle conversion failed for a portion of data.")

    return euler_angles


def process_and_save_group(group_data, seq_id, sensor_cols, preprocessed_dir, target_seq_len):
    """
    处理单个序列的 Polars DataFrame，统一长度，然后保存为 .npy。
    这个函数是为并行化设计的。
    """
    # 1. 选择最终需要的列并填充空值
    processed_df = group_data.select(sensor_cols).fill_null(0)

    # 2. ToF=-1 to 255 的处理 (已在 Polars 中完成，这里不需要)

    # 3. 转换为 Numpy 数组
    ts_data = processed_df.to_numpy() # Shape: (L, C)

    # 4. 统一序列长度
    current_seq_len = ts_data.shape[0]
    if current_seq_len < target_seq_len:
        # 如果序列太短，用0填充
        padding_shape = (target_seq_len - current_seq_len, ts_data.shape[1])
        padding = np.zeros(padding_shape, dtype=ts_data.dtype)
        final_data = np.vstack((ts_data, padding))
    else:
        # 如果序列太长，从开头截断
        final_data = ts_data[:target_seq_len, :]

    # 5. 转置以匹配 (C, L) 的格式并保存
    ts_data_processed = final_data.T
    np.save(os.path.join(preprocessed_dir, f"{seq_id}.npy"), ts_data_processed)

# =============================================================================
# 主预处理逻辑
# =============================================================================

# 假设 CFG 已经定义好
# CFG.RUN_PREPROCESSING, CFG.PREPROCESSED_DIR, CFG.TRAIN_CSV, CFG.TARGET_SEQ_LEN etc.

if CFG.RUN_PREPROCESSING:
    # 1. 清理旧目录
    if os.path.exists(CFG.PREPROCESSED_DIR):
        print(f"Removing old preprocessed data at: {CFG.PREPROCESSED_DIR}")
        shutil.rmtree(CFG.PREPROCESSED_DIR)
    os.makedirs(CFG.PREPROCESSED_DIR, exist_ok=True)

    # 2. 加载原始CSV (使用 Polars 的 Lazy API 以节省内存)
    print("Reading original CSV using Polars LazyFrame...")
    lazy_df = pl.scan_csv(CFG.TRAIN_CSV)

    # 3. 特征工程 (所有操作都在 Polars LazyFrame 上定义)
    print("Defining feature engineering steps...")

    # 处理 ToF 缺失值
    tof_cols = [c for c in lazy_df.columns if 'tof_' in c]
    if tof_cols:
        replace_exprs = [pl.when(pl.col(c) == -1).then(255).otherwise(pl.col(c)).alias(c) for c in tof_cols]
        lazy_df = lazy_df.with_columns(replace_exprs)

    # --- 执行需要 Numpy 的特征工程 ---
    print("Executing vectorized feature engineering (Gravity, Euler)...")
    # collect() 执行计算，将 LazyFrame 物化为 DataFrame 以便与 Numpy 交互
    train_df = lazy_df.collect()

    # a. 移除重力
    acc_data = train_df.select(['acc_x', 'acc_y', 'acc_z']).to_numpy()
    rot_data = train_df.select(['rot_x', 'rot_y', 'rot_z', 'rot_w']).to_numpy()
    linear_accel_data = remove_gravity_from_acc_vectorized(acc_data, rot_data)
    linear_accel_df = pl.from_numpy(linear_accel_data, schema=['linear_acc_x', 'linear_acc_y', 'linear_acc_z'])

    # b. 计算欧拉角
    euler_angles_data = euler_from_quat_vectorized(rot_data)
    euler_df = pl.from_numpy(euler_angles_data, schema=['roll', 'pitch', 'yaw'])

     # ‼️ c. 新增：计算角速度 ‼️
    print("Calculating angular velocity per sequence...")
    train_pd = train_df.to_pandas()
    # 填充NaN值
    quat_cols = ['rot_x', 'rot_y', 'rot_z', 'rot_w']
    train_pd[quat_cols] = train_pd[quat_cols].fillna(0)

    # ‼️‼️ 修正点在这里 ‼️‼️
    def quat_diff_to_angular_vel(group):
        quat_values = group[quat_cols].values

        # 安全检查：找到零范数的四元数
        norms = np.linalg.norm(quat_values, axis=1)
        zero_norm_mask = (norms == 0)

        # 如果存在零范数，则将其替换为单位四元数 [0, 0, 0, 1]
        if np.any(zero_norm_mask):
            quat_values[zero_norm_mask] = [0, 0, 0, 1]

        try:
            q = R.from_quat(quat_values)
            angular_vel = np.zeros((len(group), 3))
            if len(group) > 1:
                # 计算差分旋转并提取角速度向量
                # q[1:] * q[:-1].inv() -> 从 t-1 到 t 的旋转
                # .as_rotvec() -> 将该旋转表示为 (轴*角度) 的向量
                angular_vel[1:] = (q[1:] * q[:-1].inv()).as_rotvec()
            return pd.DataFrame(angular_vel, index=group.index, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'])
        except ValueError as e:
            # 添加一个备用错误处理，以防还有其他问题
            print(f"Warning: Could not process quaternions for a group in sequence {group['sequence_id'].iloc[0]}. Error: {e}. Filling with zeros.")
            return pd.DataFrame(np.zeros((len(group), 3)), index=group.index, columns=['angular_vel_x', 'angular_vel_y', 'angular_vel_z'])


    angular_vel_list = [quat_diff_to_angular_vel(group) for _, group in tqdm(train_pd.groupby('sequence_id'), desc="Angular Velocity")]
    angular_vel_pd = pd.concat(angular_vel_list)
    angular_vel_df = pl.from_pandas(angular_vel_pd)

    # 将新特征合并回主 DataFrame
    train_df = train_df.hstack(linear_accel_df).hstack(euler_df).hstack(angular_vel_df)

    # --- 在 Polars 中继续进行其他特征工程 ---
    print("Engineering Mag, Jerk, and Rolling features in Polars...")

    # 定义窗口大小
    WINDOW_SIZE = 10

    # 计算模 (Magnitude)
    train_df = train_df.with_columns([
        (pl.col('linear_acc_x')**2 + pl.col('linear_acc_y')**2 + pl.col('linear_acc_z')**2).sqrt().alias('linear_acc_mag'),
        (pl.col('rot_x')**2 + pl.col('rot_y')**2 + pl.col('rot_z')**2).sqrt().alias('rot_vel_mag')
    ])

    # 计算Jerk (加加速度) - 按序列分组计算差分
    jerk_cols = ['linear_acc_x', 'linear_acc_y', 'linear_acc_z']
    train_df = train_df.with_columns([
        pl.col(c).diff().over('sequence_id').alias(f"{c.replace('linear_acc', 'jerk')}") for c in jerk_cols
    ])

    # 计算滑动窗口统计特征 (均值和标准差)
    # 筛选出要计算滚动特征的列 (例如所有加速度和陀螺仪)
    rolling_feature_cols = [c for c in train_df.columns if 'acc_' in c or 'rot_' in c or 'jerk' in c]
    train_df = train_df.with_columns([
        pl.col(c).rolling_mean(window_size=WINDOW_SIZE, min_periods=1).over('sequence_id').alias(f'{c}_rolling_mean_{WINDOW_SIZE}')
        for c in rolling_feature_cols
    ] + [
        pl.col(c).rolling_std(window_size=WINDOW_SIZE, min_periods=1).over('sequence_id').alias(f'{c}_rolling_std_{WINDOW_SIZE}')
        for c in rolling_feature_cols
    ])

    # ‼️ 新增：计算聚合统计特征 (按 sequence_id 分组) ‼️
    tof_pixel_cols = [c for c in train_df.columns if 'tof_' in c and '_v' in c]
    thm_cols = [c for c in train_df.columns if 'thm_' in c and c.endswith(tuple(map(str, range(1, 6))))]

    agg_exprs = []
    # 1. TOF 传感器内部聚合
    for i in range(1, 6):
        sensor_tof_cols = [c for c in tof_pixel_cols if f"tof_{i}_" in c]
        if sensor_tof_cols:
            # 创建一个表达式列表，用于处理 255 -> null
            cols_expr_for_agg = [pl.when(pl.col(c) == 255).then(None).otherwise(pl.col(c)) for c in sensor_tof_cols]

            # 计算横向均值
            mean_expr = pl.mean_horizontal(cols_expr_for_agg)

            # ‼️‼️ 修正点在这里：手动实现横向标准差 ‼️‼️
            # std = sqrt( mean(x^2) - (mean(x))^2 )
            mean_of_squares_expr = pl.mean_horizontal([e**2 for e in cols_expr_for_agg])
            variance_expr = mean_of_squares_expr - mean_expr**2
            # 加一个小的 epsilon 防止开方负数
            std_expr = variance_expr.clip(lower_bound=1e-8).sqrt()

            agg_exprs.extend([
                mean_expr.alias(f'tof_{i}_mean'),
                std_expr.alias(f'tof_{i}_std'), # <-- 使用我们手动构造的表达式
                pl.min_horizontal(cols_expr_for_agg).alias(f'tof_{i}_min'),
                pl.max_horizontal(cols_expr_for_agg).alias(f'tof_{i}_max'),
            ])

    # 执行第一批聚合计算
    train_df = train_df.with_columns(agg_exprs)

    # 2. 跨传感器聚合
    tof_mean_cols = [f'tof_{i}_mean' for i in range(1, 6)]
    thm_cols = [f'thm_{i}' for i in range(1, 6)] # 确保 thm_cols 也是正确的

    # ‼️‼️ 同样需要手动实现跨传感器的横向标准差 ‼️‼️
    tof_mean_across_expr = pl.mean_horizontal(tof_mean_cols)
    tof_var_across_expr = pl.mean_horizontal([pl.col(c)**2 for c in tof_mean_cols]) - tof_mean_across_expr**2
    tof_std_across_expr = tof_var_across_expr.clip(lower_bound=1e-8).sqrt()

    thm_mean_across_expr = pl.mean_horizontal(thm_cols)
    thm_var_across_expr = pl.mean_horizontal([pl.col(c)**2 for c in thm_cols]) - thm_mean_across_expr**2
    thm_std_across_expr = thm_var_across_expr.clip(lower_bound=1e-8).sqrt()

    train_df = train_df.with_columns([
        (pl.max_horizontal(tof_mean_cols) - pl.min_horizontal(tof_mean_cols)).alias('tof_range_across_sensors'),
        tof_std_across_expr.alias('tof_std_across_sensors'), # <-- 使用新表达式
        (pl.max_horizontal(thm_cols) - pl.min_horizontal(thm_cols)).alias('thm_range_across_sensors'),
        thm_std_across_expr.alias('thm_std_across_sensors'), # <-- 使用新表达式
    ])

    print("Feature engineering complete.")

    # 4. 定义并保存最终的特征列列表
    # ‼️ SENSOR_COLS 现在要包含 angular_vel ‼️
    SENSOR_COLS = sorted([col for col in train_df.columns if any(keyword in col for keyword in [
        'acc_', 'thm_', 'tof_', 'rot_', 'linear_', 'jerk', 'roll', 'pitch', 'yaw', 'angular_vel' # <-- 新增
    ])])

    # ‼️ 定义新的聚合特征列，这些将保存到 metadata 中 ‼️
    AGGREGATE_COLS = [c for c in train_df.columns if 'tof_' in c and '_v' not in c] + \
                     [c for c in train_df.columns if 'thm_range' in c or 'thm_std' in c]
    AGGREGATE_COLS = sorted(list(set(AGGREGATE_COLS))) # 去重并排序

    print(f"Total unique timeseries features for .npy: {len(SENSOR_COLS)}")
    print(f"Total unique aggregate static features: {len(AGGREGATE_COLS)}")


    # 保存列名列表，供后续训练和推理使用
    cols_path = os.path.join(CFG.PREPROCESSED_DIR, "sensor_columns.json")
    with open(cols_path, 'w') as f:
        json.dump(SENSOR_COLS, f)
    print(f"Saved sensor column list to {cols_path}")

    # ‼️ 5. 提取聚合特征的均值，并准备保存 .npy 文件 ‼️
    print("Calculating mean of aggregate features and preparing to save .npy files...")
    # 计算每个序列的聚合特征的均值
    agg_features_mean = train_df.group_by('sequence_id').agg(
        [pl.mean(col).alias(col) for col in AGGREGATE_COLS]
    ).sort('sequence_id')

    # 将聚合特征保存到单独的文件中，以便在下一步加载
    agg_features_path = os.path.join(CFG.PREPROCESSED_DIR, "aggregate_features.parquet")
    agg_features_mean.write_parquet(agg_features_path)
    print(f"Saved aggregate features to {agg_features_path}")

    # =========================================================================
    # ‼️ 新增代码块 (已修正): 计算每个序列的全局统计特征 (Mean, Std, Skew, Kurtosis) ‼️
    # =========================================================================
    print("Calculating sequence-level statistics (mean, std, skew, kurtosis)...")

    # 1. 选择要为其计算全局统计特征的列
    cols_for_sequence_stats = [
        'linear_acc_mag', 'rot_vel_mag', 'jerk_x', 'jerk_y', 'jerk_z',
        'roll', 'pitch', 'yaw',
        'tof_std_across_sensors', 'thm_range_across_sensors'
    ]
    cols_for_sequence_stats = [c for c in cols_for_sequence_stats if c in train_df.columns]
    print(f"  - Aggregating over {len(cols_for_sequence_stats)} selected core features.")

    # 2. 构建 Polars 聚合表达式 (‼️‼️ 使用正确的语法 ‼️‼️)
    sequence_agg_exprs = []
    for col in cols_for_sequence_stats:
        sequence_agg_exprs.extend([
            # 这是正确的语法: pl.col(col).aggregation_method()
            pl.col(col).mean().alias(f'{col}_seq_mean'),
            pl.col(col).std().alias(f'{col}_seq_std'),
            pl.col(col).skew(bias=True).alias(f'{col}_seq_skew'),       # bias=True 匹配 pandas 的默认行为
            pl.col(col).kurtosis(fisher=False).alias(f'{col}_seq_kurtosis') # fisher=False 得到原始峰度
        ])

    # 3. 执行聚合计算
    sequence_stats_df = train_df.group_by('sequence_id').agg(
        sequence_agg_exprs
    ).sort('sequence_id').fill_null(0)

    # 4. 将这些新的序列级统计特征保存到单独的文件中
    sequence_stats_path = os.path.join(CFG.PREPROCESSED_DIR, "sequence_stats.parquet")
    sequence_stats_df.write_parquet(sequence_stats_path)
    print(f"Saved sequence-level statistics to {sequence_stats_path}")


    # group_by 返回一个迭代器，我们可以直接用于并行处理
    groups_iterator = train_df.group_by("sequence_id", maintain_order=True)

    Parallel(n_jobs=-1)(
        delayed(process_and_save_group)(
            group_df,
            seq_id[0],
            SENSOR_COLS,
            CFG.PREPROCESSED_DIR,
            CFG.TARGET_SEQ_LEN
        ) for seq_id, group_df in tqdm(groups_iterator, desc="Dispatching save jobs", total=train_df['sequence_id'].n_unique())
    )

    print("Pre-processing finished successfully.")

else:
    print("Skipping pre-processing as RUN_PREPROCESSING is False.")

Reading original CSV using Polars LazyFrame...
Defining feature engineering steps...
Executing vectorized feature engineering (Gravity, Euler)...


/tmp/ipython-input-7-2832950027.py:134: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  tof_cols = [c for c in lazy_df.columns if 'tof_' in c]


Calculating angular velocity per sequence...


Angular Velocity:   0%|          | 0/8151 [00:00<?, ?it/s]

Engineering Mag, Jerk, and Rolling features in Polars...


/tmp/ipython-input-7-2832950027.py:217: DeprecationWarning: the argument `min_periods` for `Expr.rolling_mean` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  pl.col(c).rolling_mean(window_size=WINDOW_SIZE, min_periods=1).over('sequence_id').alias(f'{c}_rolling_mean_{WINDOW_SIZE}')
/tmp/ipython-input-7-2832950027.py:220: DeprecationWarning: the argument `min_periods` for `Expr.rolling_std` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  pl.col(c).rolling_std(window_size=WINDOW_SIZE, min_periods=1).over('sequence_id').alias(f'{c}_rolling_std_{WINDOW_SIZE}')


Feature engineering complete.
Total unique timeseries features for .npy: 400
Total unique aggregate static features: 24
Saved sensor column list to /kaggle/working/preprocessed_data/sensor_columns.json
Calculating mean of aggregate features and preparing to save .npy files...
Saved aggregate features to /kaggle/working/preprocessed_data/aggregate_features.parquet
Calculating sequence-level statistics (mean, std, skew, kurtosis)...
  - Aggregating over 10 selected core features.
Saved sequence-level statistics to /kaggle/working/preprocessed_data/sequence_stats.parquet


Dispatching save jobs:   0%|          | 0/8151 [00:00<?, ?it/s]

### Cell4

In [ ]:
# ====================================================
# 4. METADATA PREPARATION (USING HOLD-OUT VALIDATION)
# ====================================================

from sklearn.model_selection import GroupShuffleSplit
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
import json
import os
import numpy as np
import pandas as pd
import torch
import joblib

# 1. 加载元信息
print("Preparing metadata (sequence_map)...")
train_df_meta = pd.read_csv(CFG.TRAIN_CSV, usecols=['sequence_id', 'subject', 'gesture'])
train_demo = pd.read_csv(CFG.TRAIN_DEMO_CSV)

# 2. 创建和处理 sequence_map
le = LabelEncoder()
train_df_meta['gesture_encoded'] = le.fit_transform(train_df_meta['gesture'])
CFG.NUM_CLASSES = len(le.classes_)
CFG.CLASS_NAMES = list(le.classes_)  # 保存类名以供参考

# 创建二分类标签用于分层损失
print("Creating binary target labels for hierarchical loss...")
TARGET_GESTURES = [
    'Above ear - Pull hair', 'Forehead - Pull hairline', 'Forehead - Scratch',
    'Eyebrow - Pull hair', 'Eyelash - Pull hair', 'Neck - Pinch skin',
    'Neck - Scratch', 'Cheek - Pinch skin'
]
# 将手势名称映射到其编码值
target_gesture_codes = [i for i, name in enumerate(CFG.CLASS_NAMES) if name in TARGET_GESTURES]
print(f"Identified {len(target_gesture_codes)} target gesture codes: {target_gesture_codes}")

sequence_map = train_df_meta[['sequence_id', 'subject', 'gesture_encoded']].drop_duplicates()
# 添加二分类目标列 (1=目标手势, 0=非目标手势)
sequence_map['binary_target'] = sequence_map['gesture_encoded'].isin(target_gesture_codes).astype(int)
print("\nBinary target distribution (1=Target, 0=Non-Target):")
print(sequence_map['binary_target'].value_counts())

# 合并人口统计学信息
sequence_map = sequence_map.merge(train_demo, on='subject', how='left')

# 加载并合并聚合特征
agg_features_path = os.path.join(CFG.PREPROCESSED_DIR, "aggregate_features.parquet")
if os.path.exists(agg_features_path):
    print(f"\nLoading aggregate features from {agg_features_path}...")
    agg_features_df = pd.read_parquet(agg_features_path)
    sequence_map = sequence_map.merge(agg_features_df, on='sequence_id', how='left')
    sequence_map.fillna(0, inplace=True)  # 填充缺失值
    print("Aggregate features successfully merged.")
else:
    print("Warning: Aggregate features file not found. Skipping merge.")

# 处理人口统计学特征中的空值
for col in ['height_cm', 'shoulder_to_wrist_cm', 'elbow_to_wrist_cm']:
    if col in sequence_map.columns:
        sequence_map[col].fillna(sequence_map[col].mean(), inplace=True)

# 计算新的特征比率
sequence_map['height_cm_safe'] = sequence_map['height_cm'].replace(0, 1e-6)
sequence_map['shoulder_wrist_ratio'] = sequence_map['shoulder_to_wrist_cm'] / sequence_map['height_cm_safe']
sequence_map['elbow_wrist_ratio'] = sequence_map['elbow_to_wrist_cm'] / sequence_map['height_cm_safe']
sequence_map.drop('height_cm_safe', axis=1, inplace=True)

# 更新 CFG.DEMO_COLS 以包含聚合特征
AGGREGATE_COLS = sorted([
    c for c in sequence_map.columns
    if ('tof_' in c and '_v' not in c) or ('thm_range' in c or 'thm_std' in c)
])
CFG.ALL_STATIC_COLS = CFG.DEMO_COLS + AGGREGATE_COLS
print(f"Total static features (demographics + aggregates): {len(CFG.ALL_STATIC_COLS)}")

# 标准化数值列
scaler = StandardScaler()
numerical_static_cols = [
    col for col in CFG.ALL_STATIC_COLS
    if col in sequence_map.columns and sequence_map[col].dtype in ['int64', 'float64']
]
if numerical_static_cols:
    sequence_map[numerical_static_cols] = scaler.fit_transform(sequence_map[numerical_static_cols])
    print(f"Saving scaler for columns: {numerical_static_cols}")
    os.makedirs(CFG.MODEL_OUTPUT_DIR, exist_ok=True)

    # 保存标准化器
    scaler_path = os.path.join(CFG.MODEL_OUTPUT_DIR, "min_max_scaler.joblib")
    joblib.dump(scaler, scaler_path)
    print(f"Scaler successfully saved to: {scaler_path}")

    # 保存标准化的列名
    scaler_cols_path = os.path.join(CFG.MODEL_OUTPUT_DIR, "scaler_columns.json")
    with open(scaler_cols_path, 'w') as f:
        json.dump(numerical_static_cols, f)
    print(f"Scaler columns saved to: {scaler_cols_path}")

# 计算类别权重以处理类别不平衡
print("\nCalculating class weights to handle imbalance...")
class_labels = np.unique(sequence_map['gesture_encoded'])
class_weights_array = compute_class_weight(
    'balanced',
    classes=class_labels,
    y=sequence_map['gesture_encoded'].values
)
CLASS_WEIGHTS = torch.tensor(class_weights_array, dtype=torch.float32).to(CFG.DEVICE)
print(f"Class weights calculated and moved to {CFG.DEVICE}.")
print(f"Weights: {CLASS_WEIGHTS}")

# 创建 K-Fold 分割
print(f"\nCreating {CFG.N_FOLDS}-fold splits, grouped by subject and stratified by gesture...")
skf = StratifiedGroupKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=42)
sequence_map['fold'] = -1
for fold_num, (train_idx, val_idx) in enumerate(skf.split(sequence_map, sequence_map['gesture_encoded'], sequence_map['subject'])):
    sequence_map.loc[val_idx, 'fold'] = fold_num

# 打印分割分布
print("Fold distribution:")
print(sequence_map['fold'].value_counts())
assert (sequence_map['fold'] == -1).sum() == 0, "Some sequences were not assigned a fold!"

### CELL 5

In [ ]:
# ====================================================
# CELL 5: LEARNING RATE FINDER (ADAPTED FOR HIERARCHICAL LOSS)
# ====================================================

import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import numpy as np
from tqdm.notebook import tqdm


class LRFinder:
    """
    一个适配了分层损失的学习率范围测试器。
    """
    def __init__(self, model, optimizer, criterion_18_class, criterion_binary, loss_alpha, device):
        """
        初始化学习率范围测试器。

        Args:
            model: 你的 PyTorch 模型。
            optimizer: 你的优化器。
            criterion_18_class: 用于18分类任务的损失函数。
            criterion_binary: 用于二分类任务的损失函数。
            loss_alpha (float): 二分类损失的权重。
            device: 'cuda' 或 'cpu'。
        """
        self.model = model
        self.optimizer = optimizer
        self.criterion_18_class = criterion_18_class
        self.criterion_binary = criterion_binary
        self.loss_alpha = loss_alpha
        self.device = device
        self.history = {"lr": [], "loss": []}

        # 保存模型初始状态，以便测试后恢复
        torch.save(model.state_dict(), 'lr_finder_initial_state.pth')

    def range_test(self, train_loader, start_lr=1e-7, end_lr=1, num_iter=100, beta=0.98):
        """
        执行学习率范围测试。

        Args:
            train_loader: 训练数据加载器。
            start_lr (float): 起始学习率。
            end_lr (float): 结束学习率。
            num_iter (int): 测试的迭代次数。
            beta (float): 平滑参数。
        """
        self.model.train()
        lr_scheduler = LRScheduler(self.optimizer, start_lr, end_lr, num_iter)
        avg_loss = 0.0

        data_iter = iter(train_loader)
        pbar = tqdm(range(num_iter), desc="LR Range Test (Hierarchical Loss)")

        for i in pbar:
            try:
                # 解包5个项目
                imu, thermo, static, labels_18, labels_binary = next(data_iter)
            except StopIteration:
                data_iter = iter(train_loader)
                imu, thermo, static, labels_18, labels_binary = next(data_iter)

            # 将数据移动到设备
            imu, thermo, static, labels_18, labels_binary = (
                imu.to(self.device),
                thermo.to(self.device),
                static.to(self.device),
                labels_18.to(self.device),
                labels_binary.to(self.device)
            )

            # 前向传播，模型返回两组输出
            outputs_18, outputs_binary = self.model(imu, thermo, static)
            outputs_18_float = outputs_18_half.float()
            outputs_binary_float = outputs_binary_half.float()

            # 计算分层组合损失
            loss18 = self.criterion_18_class(outputs_18_float, labels_18)
            loss2 = self.criterion_binary(outputs_binary_float, labels_binary)
            loss = (1 - self.loss_alpha) * loss18 + self.loss_alpha * loss2

            # 计算平滑损失
            avg_loss = beta * avg_loss + (1 - beta) * loss.item()
            smoothed_loss = avg_loss / (1 - beta**(i + 1))

            self.history["lr"].append(lr_scheduler.get_lr()[0])
            self.history["loss"].append(smoothed_loss)

            pbar.set_postfix(loss=smoothed_loss, lr=lr_scheduler.get_lr()[0])

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            lr_scheduler.step()

            if i > 0 and smoothed_loss > 4 * self.history["loss"][0]:
                print("\nLoss diverged. Stopping early.")
                break

        print("\nLR Range Test Finished.")

    def plot(self, skip_start=10, skip_end=5, suggestion=None):
        """
        绘制学习率范围测试结果。

        Args:
            skip_start (int): 跳过开始的若干点。
            skip_end (int): 跳过结尾的若干点。
            suggestion (float): 建议的学习率（可选）。
        """
        if not self.history["lr"]:
            print("No history to plot. Please run range_test first.")
            return

        lrs = self.history["lr"][skip_start:-skip_end]
        losses = self.history["loss"][skip_start:-skip_end]

        plt.figure(figsize=(10, 6))
        plt.plot(lrs, losses)
        if suggestion:
            plt.axvline(x=suggestion, color='r', linestyle='--', label=f'Suggested LR: {suggestion:.2e}')
            plt.legend()
        plt.xscale("log")
        plt.xlabel("Learning Rate (log scale)")
        plt.ylabel("Combined Loss")
        plt.title("Learning Rate Range Test")
        plt.grid(True, which="both", ls="--")
        plt.show()

    def reset(self):
        """
        恢复模型到其初始状态。
        """
        self.model.load_state_dict(torch.load('lr_finder_initial_state.pth'))
        print("Model state has been reset to its initial state.")

    def suggestion(self, skip_start=10, skip_end=5):
        """
        根据梯度变化建议学习率。

        Args:
            skip_start (int): 跳过开始的若干点。
            skip_end (int): 跳过结尾的若干点。

        Returns:
            float: 建议的学习率，若无建议则返回 None。
        """
        if not self.history["lr"] or len(self.history["lr"]) < skip_start + skip_end + 2:
            print("Not enough history to suggest a learning rate.")
            return None

        lrs = self.history["lr"][skip_start:-skip_end]
        losses = self.history["loss"][skip_start:-skip_end]
        log_lrs = np.log10(lrs)
        gradients = np.gradient(losses, log_lrs)
        min_grad_idx = np.argmin(gradients)
        return lrs[min_grad_idx]


# ====================================================
# LRScheduler 类 (保持不变)
# ====================================================

class LRScheduler:
    """
    学习率调度器，用于线性或指数增加学习率。
    """
    def __init__(self, optimizer, start_lr, end_lr, num_iter):
        self.optimizer = optimizer
        self.start_lr = start_lr
        self.end_lr = end_lr
        self.num_iter = num_iter
        self.iter = 0
        self.factor = (end_lr / start_lr) ** (1 / (num_iter - 1))

    def step(self):
        """
        更新当前学习率。
        """
        self.iter += 1
        lr = self.start_lr * (self.factor ** self.iter)
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr

    def get_lr(self):
        """
        获取当前学习率。
        """
        return [param_group['lr'] for param_group in self.optimizer.param_groups]

In [ ]:
# ===================================================='
# CELL 7: DATASET AND MODEL DEFINITION (ADVANCED DUAL-TOWER)
# ===================================================='
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import numpy as np
import os
import torch.nn.functional as F


# ===================================================='
# 完整的、优化后的 Dataset 类
# ===================================================='

class DualTowerDataset(Dataset):
    """
    一个高效的数据集类，它加载预处理好的 .npy 文件。
    ‼️ 已修改: 返回18分类和二分类两种标签，用于分层损失。
    """
    def __init__(self, df_map, all_sensor_cols, static_cols=None, preprocessed_dir=CFG.PREPROCESSED_DIR, is_train=False):
        self.df_map = df_map.reset_index(drop=True)
        self.static_cols = static_cols
        self.preprocessed_dir = preprocessed_dir
        self.is_train = is_train

        imu_keywords = ['acc_', 'rot_', 'linear_', 'jerk', 'roll', 'pitch', 'yaw', 'angular_vel']
        self.imu_indices = [i for i, col in enumerate(all_sensor_cols) if any(keyword in col for keyword in imu_keywords)]

        all_indices_set = set(range(len(all_sensor_cols)))
        self.thermo_tof_indices = sorted(list(all_indices_set - set(self.imu_indices)))

        self.num_imu_channels = len(self.imu_indices)
        self.num_thermo_tof_channels = len(self.thermo_tof_indices)

        print(f"Dataset created. Mode: {'Train' if is_train else 'Validation'}.")
        print(f"  - IMU Tower Channels: {self.num_imu_channels}")
        print(f"  - Thermo/ToF Tower Channels: {self.num_thermo_tof_channels}")
        assert (self.num_imu_channels + self.num_thermo_tof_channels) == len(all_sensor_cols), "Mismatch in total channels!"

    def __len__(self):
        return len(self.df_map)

    def _apply_augmentations(self, data_array):
        if np.random.rand() < 0.3:
            data_array += np.random.normal(0, 0.01, data_array.shape)
        if np.random.rand() < 0.3:
            data_array *= np.random.uniform(0.9, 1.1)
        if np.random.rand() < 0.3:
            max_shift = int(data_array.shape[1] * 0.1)
            shift_amount = np.random.randint(-max_shift, max_shift + 1)
            if shift_amount != 0:
                data_array = np.roll(data_array, shift_amount, axis=1)
        return data_array

    def __getitem__(self, idx):
        seq_info = self.df_map.iloc[idx]
        seq_id = seq_info['sequence_id']
        # ‼️ 修改: 获取18分类和二分类两种标签
        label_18 = seq_info['gesture_encoded']
        label_binary = seq_info['binary_target']

        file_path = os.path.join(self.preprocessed_dir, f"{seq_id}.npy")
        ts_data = np.load(file_path)

        imu_data = ts_data[self.imu_indices, :]
        thermo_tof_data = ts_data[self.thermo_tof_indices, :]

        if self.is_train:
            if np.random.rand() < 0.5:
                thermo_tof_data = np.zeros_like(thermo_tof_data)
            imu_data = self._apply_augmentations(imu_data)

        imu_tensor = torch.tensor(imu_data, dtype=torch.float32)
        thermo_tof_tensor = torch.tensor(thermo_tof_data, dtype=torch.float32)
        static_features = torch.tensor(seq_info[self.static_cols].values.astype(np.float32), dtype=torch.float32)

        # ‼️ 修改: 将两种标签都转换为Tensor并返回
        return imu_tensor, thermo_tof_tensor, static_features, torch.tensor(label_18, dtype=torch.long), torch.tensor(label_binary, dtype=torch.long)



import torch
import torch.nn as nn
import torch.nn.functional as F


import torch
import torch.nn as nn
import torch.nn.functional as F

# ====================================================
# CELL 1: CORE BUILDING BLOCKS (Largely unchanged, but with docstrings)
# ====================================================

class SEModule(nn.Module):
    """Squeeze-and-Excitation Module for 1D convolutions."""
    def __init__(self, channels, reduction=16):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(channels, channels // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channels // reduction, channels, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1)
        return x * y.expand_as(x)

class ResidualBlockSE(nn.Module):
    """Residual Block with a Squeeze-and-Excitation module."""
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, downsample=None):
        super(ResidualBlockSE, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding=(kernel_size - 1) // 2, bias=False)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding=(kernel_size - 1) // 2, bias=False)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.se = SEModule(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        if self.downsample is not None:
            identity = self.downsample(x)
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.se(out)
        out += identity
        return self.relu(out)

class GeM(nn.Module):
    """Generalized Mean Pooling."""
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return F.avg_pool1d(x.clamp(min=self.eps).pow(self.p), x.size(-1)).pow(1./self.p)

# ====================================================
# CELL 2: CONFORMER BLOCKS (Central component of the new tower)
# ====================================================

class Swish(nn.Module):
    """Swish activation function."""
    def forward(self, x):
        return x * torch.sigmoid(x)

class Transpose(nn.Module):
    """A simple layer to transpose dimensions."""
    def __init__(self, dim0, dim1):
        super().__init__()
        self.dim0, self.dim1 = dim0, dim1
    def forward(self, x):
        return x.transpose(self.dim0, self.dim1)

class FeedForwardModule(nn.Module):
    """Feed-Forward Network module from the Conformer paper."""
    def __init__(self, d_model, expansion_factor=4, dropout=0.1):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, d_model * expansion_factor),
            Swish(),
            nn.Dropout(dropout),
            nn.Linear(d_model * expansion_factor, d_model),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.sequential(x)

class ConvolutionModule(nn.Module):
    """Convolutional module from the Conformer paper."""
    def __init__(self, d_model, kernel_size=31, dropout=0.1):
        super().__init__()
        assert kernel_size % 2 == 1, "Convolution kernel size must be odd."
        self.sequential = nn.Sequential(
            nn.LayerNorm(d_model),
            Transpose(1, 2),  # (B, L, C) -> (B, C, L)
            nn.Conv1d(d_model, 2 * d_model, kernel_size=1, stride=1, padding=0),
            nn.GLU(dim=1),
            nn.Conv1d(d_model, d_model, kernel_size, stride=1, padding=(kernel_size - 1)//2, groups=d_model),
            nn.BatchNorm1d(d_model),
            Swish(),
            nn.Conv1d(d_model, d_model, kernel_size=1, stride=1, padding=0),
            nn.Dropout(dropout),
            Transpose(1, 2)  # (B, C, L) -> (B, L, C)
        )

    def forward(self, x):
        return self.sequential(x)

class ConformerBlock(nn.Module):
    """
    The full Conformer block, following the structure: FFN -> Attention -> Conv -> FFN.
    Each connection is residual.
    """
    def __init__(self, d_model, n_head, ffn_expansion_factor=4, conv_kernel_size=31, dropout=0.1):
        super().__init__()
        self.ffn1 = FeedForwardModule(d_model, ffn_expansion_factor, dropout)
        self.attention_norm = nn.LayerNorm(d_model)
        self.attention = nn.MultiheadAttention(d_model, n_head, dropout=dropout, batch_first=True)
        self.attention_dropout = nn.Dropout(dropout)
        self.conv_module = ConvolutionModule(d_model, conv_kernel_size, dropout)
        self.ffn2 = FeedForwardModule(d_model, ffn_expansion_factor, dropout)
        self.final_norm = nn.LayerNorm(d_model)

    def forward(self, x):
        # Input x shape: (batch, seq_len, d_model)
        x = x + 0.5 * self.ffn1(x)
        residual = x
        x = self.attention_norm(x)
        attn_output, _ = self.attention(x, x, x)
        x = residual + self.attention_dropout(attn_output)
        x = x + self.conv_module(x)
        x = x + 0.5 * self.ffn2(x)
        x = self.final_norm(x)
        return x

# =========================================================
# CELL 3: OPTIMIZED SENSOR TOWER AND FUSION
# =========================================================

class SensorConformerTower(nn.Module):
    """
    OPTIMIZED: A generic and configurable tower for time-series sensor data.
    It uses a deep CNN backbone for downsampling and local feature extraction,
    followed by a stack of Conformer blocks to model long-range dependencies.
    """
    def __init__(self, in_channels, embed_dim=192, cnn_depth=2, n_layers=4, n_head=4, dropout=0.1):
        super().__init__()

        # --- Deep CNN Backbone ---
        layers = []
        # Initial convolution layer
        layers.extend([
            nn.Conv1d(in_channels, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool1d(kernel_size=3, stride=2, padding=1)
        ])

        # Stack of residual blocks
        current_channels = 64
        for i in range(cnn_depth):
            # Downsample on the first block
            stride = 2 if i == 0 else 1
            downsample = nn.Sequential(
                nn.Conv1d(current_channels, embed_dim, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(embed_dim)
            ) if i == 0 or current_channels != embed_dim else None

            layers.append(ResidualBlockSE(
                current_channels, embed_dim, kernel_size=3, stride=stride, downsample=downsample
            ))
            current_channels = embed_dim

        self.backbone = nn.Sequential(*layers)

        # --- Conformer Encoder ---
        self.conformer_encoder = nn.Sequential(
            *[ConformerBlock(d_model=embed_dim, n_head=n_head, dropout=dropout) for _ in range(n_layers)]
        )

        self.pooling = GeM()
        self.flatten = nn.Flatten()

    def forward(self, x):
        """
        Input x: (Batch, Channels, TimeSteps)
        """
        # 1. CNN Backbone for local features and downsampling
        x = self.backbone(x)  # -> (B, embed_dim, L_out)

        # 2. Reshape for Conformer: (B, C, L) -> (B, L, C)
        x = x.permute(0, 2, 1)

        # 3. Conformer for global context
        x = self.conformer_encoder(x)  # -> (B, L_out, embed_dim)

        # 4. Reshape for pooling: (B, L, C) -> (B, C, L)
        x = x.permute(0, 2, 1)

        # 5. Pool and flatten to get a fixed-size vector
        x = self.pooling(x)    # -> (B, embed_dim, 1)
        x = self.flatten(x)    # -> (B, embed_dim)

        return x


class EnhancedFusionBlock(nn.Module):
    """
    OPTIMIZED: A more powerful fusion block that allows for rich, non-linear
    interaction between the features from different modalities.
    """
    def __init__(self, combined_dim, hidden_dim_multiplier=2, dropout=0.1):
        super().__init__()
        hidden_dim = combined_dim * hidden_dim_multiplier
        self.fusion_net = nn.Sequential(
            nn.LayerNorm(combined_dim),
            nn.Linear(combined_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, combined_dim)
        )

    def forward(self, combined_features):
        # The residual connection is crucial for stable training
        return combined_features + self.fusion_net(combined_features)


# =========================================================
# CELL 4: FINAL OPTIMIZED MODEL
# =========================================================

class OptimizedDualTowerModel(nn.Module):
    """
    OPTIMIZED main model with architectural symmetry, enhanced fusion, and better configuration.

    This model has a shared backbone and two separate classifier heads for hierarchical loss.
    """
    def __init__(self,
                 num_imu_channels: int,
                 num_thermo_tof_channels: int,
                 num_static_features: int,
                 num_classes: int,
                 # --- Model Configuration ---
                 embed_dim: int = 192,
                 cnn_depth: int = 2,
                 n_layers_conformer: int = 4,
                 n_head: int = 4,
                 dropout: float = 0.2):
        super().__init__()

        # --- Tower 1: IMU Sensor Data ---
        self.imu_tower = SensorConformerTower(
            in_channels=num_imu_channels,
            embed_dim=embed_dim,
            cnn_depth=cnn_depth,
            n_layers=n_layers_conformer,
            n_head=n_head,
            dropout=dropout
        )

        # --- Tower 2: Thermo/ToF Sensor Data (Symmetric Architecture) ---
        self.thermo_tof_tower = SensorConformerTower(
            in_channels=num_thermo_tof_channels,
            embed_dim=embed_dim,
            cnn_depth=cnn_depth,
            n_layers=n_layers_conformer,
            n_head=n_head,
            dropout=dropout
        )

        # --- Fusion and Classification ---
        combined_dim = embed_dim + embed_dim + num_static_features

        # Use the new, more powerful fusion block
        self.fusion_block = EnhancedFusionBlock(combined_dim, dropout=dropout)

        # Shared classifier backbone
        self.classifier_backbone = nn.Sequential(
            nn.LayerNorm(combined_dim), # Use LayerNorm for batch-independence
            nn.Linear(combined_dim, 512),
            nn.GELU(), # GELU is often a good alternative to ReLU
            nn.Dropout(0.3) # Slightly higher dropout before final layers
        )

        # Heads for hierarchical loss
        self.classifier_18_class = nn.Linear(512, num_classes)
        self.classifier_binary = nn.Linear(512, 2)

    def forward(self, imu_data, thermo_tof_data, static_features):
        # 1. Process each modality through its dedicated tower
        imu_features = self.imu_tower(imu_data)             # (B, embed_dim)
        thermo_tof_features = self.thermo_tof_tower(thermo_tof_data) # (B, embed_dim)

        # 2. Combine features from all sources
        # Ensure static_features is (B, num_static_features)
        combined_features = torch.cat([imu_features, thermo_tof_features, static_features], dim=1)

        # 3. Fuse the combined features with the enhanced block
        fused_features = self.fusion_block(combined_features)

        # 4. Pass through shared classifier backbone and then to heads
        shared_output = self.classifier_backbone(fused_features)
        logits_18_class = self.classifier_18_class(shared_output)
        logits_binary = self.classifier_binary(shared_output)

        return logits_18_class, logits_binary

### CELL 6

In [ ]:
# ====================================================
# 6. TRAINING LOOP (SIMPLIFIED & ADAPTED)
# ====================================================

from torch.utils.data import WeightedRandomSampler


def load_preprocessed_sensor_cols(cfg):
    """
    加载工程化的传感器列列表。
    """
    cols_path = os.path.join(cfg.PREPROCESSED_DIR, "sensor_columns.json")
    if not os.path.exists(cols_path):
        raise FileNotFoundError(f"关键产物文件未找到: {cols_path}。请先运行预处理。")
    print(f"从 {cols_path} 加载权威的传感器列列表...")
    with open(cols_path, 'r') as f:
        sensor_cols = json.load(f)
    print(f"成功加载 {len(sensor_cols)} 个工程化特征列。")
    return sensor_cols


try:
    ALL_SENSOR_COLS = load_preprocessed_sensor_cols(CFG)
except FileNotFoundError as e:
    print(f"错误: {e}")


# ====================================================
# EarlyStopping Class (保持不变)
# ====================================================

import numpy as np
import torch

class EarlyStopping:
    """
    用于检测验证集性能的早停机制。
    """
    def __init__(self, patience=5, verbose=False, delta=0, path='best_model.pth', trace_func=print, mode='max'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_metric_best = np.Inf if mode == 'min' else -np.Inf
        self.delta = delta if mode == 'max' else -delta
        self.path = path
        self.trace_func = trace_func
        self.mode = mode

    def __call__(self, val_metric, model):
        score = val_metric if self.mode == 'max' else -val_metric
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_metric, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.counter = 0
            self.save_checkpoint(val_metric, model)

    def save_checkpoint(self, val_metric, model):
        """
        保存验证集性能最优的模型。
        """
        if self.verbose:
            self.trace_func(f'Validation metric improved ({self.val_metric_best:.6f} --> {val_metric:.6f}). Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_metric_best = val_metric




## 训练loop

In [ ]:
# ====================================================
# Training Loop
# ====================================================

from accelerate import Accelerator

# =========================================================================
# ‼️ 新增: Mixup 辅助函数 ‼️
# =========================================================================

def mixup_data(x_imu, x_thermo, x_static, y_18, y_binary, alpha=0.5, device='cuda'):
    """返回 mixed inputs, a, b, 和 lambda."""
    if alpha > 0:
        # lam 是 beta 分布的采样，决定混合比例
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x_imu.size()[0]
    # 生成随机的索引，用于将当前 batch 内的样本进行混合
    index = torch.randperm(batch_size).to(device)

    # 混合输入数据
    mixed_x_imu = lam * x_imu + (1 - lam) * x_imu[index, :]
    mixed_x_thermo = lam * x_thermo + (1 - lam) * x_thermo[index, :]
    mixed_x_static = lam * x_static + (1 - lam) * x_static[index, :]

    # 获取原始标签和被混合的标签
    y_18_a, y_18_b = y_18, y_18[index]
    y_binary_a, y_binary_b = y_binary, y_binary[index]

    return mixed_x_imu, mixed_x_thermo, mixed_x_static, y_18_a, y_18_b, y_binary_a, y_binary_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """计算 Mixup 的损失"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

# --- 定义常量 ---
MIXUP_ALPHA = 0.5 # 现在我们将真正使用它

def train_model(fold: int, accelerator: Accelerator):
    accelerator.print(f"========== STARTING TRAINING FOR FOLD {fold} (with Mixup) ==========")

    # 数据集划分和加载器 (这部分逻辑不变)
    train_map = sequence_map[sequence_map['fold'] != fold].reset_index(drop=True)
    val_map = sequence_map[sequence_map['fold'] == fold].reset_index(drop=True)
    train_dataset = DualTowerDataset(train_map, all_sensor_cols=ALL_SENSOR_COLS, static_cols=CFG.ALL_STATIC_COLS, is_train=True)
    val_dataset = DualTowerDataset(val_map, all_sensor_cols=ALL_SENSOR_COLS, static_cols=CFG.ALL_STATIC_COLS, is_train=False)

    # WeightedRandomSampler (这部分逻辑不变)
    train_targets = torch.tensor(train_dataset.df_map['gesture_encoded'].values)
    class_counts = torch.bincount(train_targets)
    class_weights = 1. / class_counts.float()
    sample_weights = torch.tensor([class_weights[t] for t in train_targets])
    sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=CFG.BATCH_SIZE, sampler=sampler, shuffle=False, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=CFG.BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=True)

    # 模型、优化器、调度器 (这部分逻辑不变)
    model = OptimizedDualTowerModel(num_imu_channels=train_dataset.num_imu_channels,
        num_thermo_tof_channels=train_dataset.num_thermo_tof_channels,
        num_static_features=len(CFG.ALL_STATIC_COLS),
        num_classes=CFG.NUM_CLASSES,
        embed_dim=200,
        cnn_depth=3,
        n_layers_conformer=2,
        n_head=4,
        dropout=0.3)
    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.LEARNING_RATE)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=CFG.LEARNING_RATE,
        steps_per_epoch=len(train_loader),
        epochs=CFG.EPOCHS,
        pct_start=0.2
    )


    # Accelerate 准备
    model, optimizer, train_loader, val_loader, scheduler = accelerator.prepare(model, optimizer, train_loader, val_loader, scheduler)

    # 损失函数
    criterion_18_class = nn.CrossEntropyLoss(label_smoothing=0.1)
    criterion_binary = nn.CrossEntropyLoss()

    # ====================================================
    # 优化的 LR FINDER 调用部分 (默认注释，需要时取消注释)
    # ====================================================

    # print("--- Starting Learning Rate Finder ---")

    # # 初始化 LRFinder，传入模型、优化器、损失函数和 alpha 参数
    # lr_finder = LRFinder(
    #     model,
    #     optimizer,
    #     criterion_18_class,
    #     criterion_binary,
    #     CFG.HIERARCHICAL_LOSS_ALPHA,
    #     device=CFG.DEVICE
    # )

    # # 执行学习率范围测试
    # lr_finder.range_test(train_loader, start_lr=1e-8, end_lr=10, num_iter=300)

    # # 绘制测试结果并获取建议的学习率
    # suggestion = lr_finder.suggestion()
    # print(f"Suggested LR: {suggestion}")
    # lr_finder.plot(skip_start=10, skip_end=5, suggestion=suggestion)

    # # 重置模型和优化器，准备正式训练
    # lr_finder.reset()
    # print("--- LR Finder Finished. Ready for main training. ---")

    # # 如果仅运行学习率测试，可以直接退出，避免执行后续训练
    # return [] , []


    # 模型保存和早停设置
    model_path = f"{CFG.MODEL_OUTPUT_DIR}/best_model_fold_{fold}.pth"
    early_stopping = EarlyStopping(patience=30, verbose=accelerator.is_main_process, path=model_path, mode='min')

    best_f1 = 0.0
    for epoch in range(CFG.EPOCHS):
        model.train()

        epoch_total_loss = 0.0

        # 使用tqdm时，通过 accelerator.is_main_process 禁用非主进程的进度条
        pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{CFG.EPOCHS} [Training]", disable=not accelerator.is_main_process)

        for i, (imu_data, thermo_tof_data, statics, labels_18, labels_binary) in enumerate(pbar):
            # ‼️ 数据已经自动移动到正确设备，无需手动 .to(device) ‼️

            # ‼️ 移除 torch.amp.autocast, Accelerate 会自动处理 ‼️
            outputs_18, outputs_binary = model(imu_data, thermo_tof_data, statics)

            # 在混合精度下，损失计算也应在 autocast 上下文中
            # 但由于模型输出已经是 fp16/bf16, 需要转回 fp32 计算损失
            loss_18_class = criterion_18_class(outputs_18.float(), labels_18)
            loss_binary_class = criterion_binary(outputs_binary.float(), labels_binary)
            loss = (1 - CFG.HIERARCHICAL_LOSS_ALPHA) * loss_18_class + CFG.HIERARCHICAL_LOSS_ALPHA * loss_binary_class

            optimizer.zero_grad()

            # ‼️ 使用 accelerator.backward(loss) 代替 loss.backward() ‼️
            # 它会自动处理梯度缩放 (GradScaler)
            accelerator.backward(loss)

            optimizer.step()
            scheduler.step()

            epoch_total_loss += loss.item()

            if accelerator.is_main_process:
                pbar.set_postfix({
                    'loss': epoch_total_loss / (i + 1),
                    'lr': scheduler.get_last_lr()[0]
                })

        # --- 验证循环 ---
        model.eval()
        val_loss = 0.0
        all_labels, all_preds = [], []

        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{CFG.EPOCHS} [Validation]", disable=not accelerator.is_main_process)

        with torch.no_grad():
            for imu_data, thermo_tof_data, statics, labels_18, labels_binary in val_pbar:
                outputs_18, outputs_binary = model(imu_data, thermo_tof_data, statics)

                loss_18_class = criterion_18_class(outputs_18.float(), labels_18)
                loss_binary_class = criterion_binary(outputs_binary.float(), labels_binary)
                loss = (1 - CFG.HIERARCHICAL_LOSS_ALPHA) * loss_18_class + CFG.HIERARCHICAL_LOSS_ALPHA * loss_binary_class
                val_loss += loss.item() * imu_data.size(0)

                preds = torch.argmax(outputs_18, dim=1)

                # ‼️ 关键: 使用 accelerator.gather_for_metrics ‼️
                # 在分布式环境中，这会从所有进程收集张量，确保评估在整个验证集上进行
                gathered_preds = accelerator.gather_for_metrics(preds)
                gathered_labels = accelerator.gather_for_metrics(labels_18)

                all_preds.append(gathered_preds.cpu())
                all_labels.append(gathered_labels.cpu())

        # 在循环外拼接所有批次的张量
        all_preds = torch.cat(all_preds).numpy()
        all_labels = torch.cat(all_labels).numpy()

        avg_val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = accuracy_score(all_labels, all_preds)
        val_f1 = f1_score(all_labels, all_preds, average='macro')

        # 只在主进程打印指标和执行早停/保存
        if accelerator.is_main_process:
            accelerator.print(f"Epoch {epoch + 1}, Val Loss: {avg_val_loss:.6f}, Val Accuracy: {val_accuracy:.6f}, Val Macro F1: {val_f1:.4f}")

            # ‼️ 保存模型时，先用 unwrap_model 获取原始的PyTorch模型 ‼️
            # 这能确保保存的模型状态字典不包含 Accelerate 的包装，方便后续加载
            unwrapped_model = accelerator.unwrap_model(model)
            early_stopping(avg_val_loss, unwrapped_model)

            if val_f1 > best_f1:
                best_f1 = val_f1
                accelerator.print(f"🚀 New best macro_f1: {best_f1:.4f}. Saving model...")
                # 同样保存解包后的模型
                accelerator.save(unwrapped_model.state_dict(), f"{CFG.MODEL_OUTPUT_DIR}/best_model_fold_{fold}_f1.pth")

        # 同步所有进程，确保一个进程的早停信号能被所有进程感知
        accelerator.wait_for_everyone()
        if early_stopping.early_stop:
            accelerator.print("Early stopping triggered. Ending training.")
            break

    # 加载最佳模型进行最终评估或返回
    if accelerator.is_main_process:
        accelerator.print(f"Training finished for fold {fold}. Best model saved at {model_path}")

    # 清理内存
    del model, optimizer, train_loader, val_loader, scheduler
    gc.collect()
    torch.cuda.empty_cache()

    return all_labels, all_preds


### CELL7

In [ ]:
def main():
    # ‼️ 1. 初始化 Accelerator ‼️
    # 可以通过 `Accelerator` 的参数配置混合精度、日志等
    # 例如: mixed_precision='fp16' 或 'bf16'
    accelerator = Accelerator(mixed_precision='fp16', log_with=None)

    # 打印 Accelerator 状态 (设备、分布式类型等)
    accelerator.print(accelerator.state)

    oof_labels = []
    oof_preds = []

    # 训练循环
    for fold in range(1): # 改为只跑一个fold进行演示
        # 将 accelerator 实例传递给训练函数
        fold_labels, fold_preds = train_model(fold=fold, accelerator=accelerator)

        # OOF 结果只在主进程上聚合
        if accelerator.is_main_process:
            oof_labels.extend(fold_labels)
            oof_preds.extend(fold_preds)
            accelerator.print(f"========== FOLD {fold} FINISHED ==========")
            accelerator.print(f"{'='*50}\n")

    # 只有主进程有完整的 OOF 结果，所以绘图和最终评估也只在主进程进行
    if accelerator.is_main_process:
        accelerator.print("\nCalculating Final Confusion Matrix for All Out-of-Fold Predictions...")

        # 类别名称列表 (请替换成你自己的)
        class_names = [f'Gesture {i}' for i in range(CFG.NUM_CLASSES)]

        cm = confusion_matrix(oof_labels, oof_preds)

        plt.figure(figsize=(12, 10))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names)
        plt.title('Final Confusion Matrix (Out-of-Fold Predictions)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        # 保存图像而不是显示，这在服务器上运行时更友好
        plt.savefig("oof_confusion_matrix.png")
        accelerator.print("Confusion matrix saved to oof_confusion_matrix.png")

        oof_accuracy = accuracy_score(oof_labels, oof_preds)
        oof_f1 = f1_score(oof_labels, oof_preds, average='macro')
        accelerator.print(f"\nFinal OOF Accuracy: {oof_accuracy:.4f}")
        accelerator.print(f"Final OOF Macro F1-Score: {oof_f1:.4f}")

if __name__ == "__main__":
    main()

In [ ]:
# # ====================================================
# # 8. CLEANUP (OPTIONAL)
# # ====================================================
# import shutil # 导入用于文件操作的库

# # 检查预处理数据目录是否存在
# if os.path.exists(CFG.PREPROCESSED_DIR):
#     print(f"Training finished. Now cleaning up the preprocessed data directory...")
#     print(f"Directory to be removed: {CFG.PREPROCESSED_DIR}")

#     try:
#         # 使用 shutil.rmtree 删除整个目录及其所有内容
#         shutil.rmtree(CFG.PREPROCESSED_DIR)
#         print(f"Successfully removed directory: {CFG.PREPROCESSED_DIR}")
#     except OSError as e:
#         print(f"Error removing directory {CFG.PREPROCESSED_DIR}: {e.strerror}")
# else:
#     print("Preprocessed data directory not found, no cleanup needed.")

# 验证一下磁盘空间是否释放 (可选)
# 在Kaggle环境中，可以使用 `!df -h` 或 `!du -sh /kaggle/working/*` 来查看
print("\nVerifying working directory contents:")
!ls -lh /kaggle/working/
